## 이름: 홍진우

## 학번: 20205276

# Neural networks with PyTorch

Pytorch의 `nn.module`을 활용하여 만드는 유용한 방법을 학습합니다.

<div style="text-align:center"><img src='https://drive.google.com/thumbnail?id=1J2SeiPpVJs1-ML2BdLrcxkGGmHpRxIVE&sz=w1000' width="250" height="200"> 

### Lego block coding! </div>

In [1]:
import torch
import numpy as np

from torch import nn
import torch.nn.functional as F
from collections import OrderedDict

torch.manual_seed(0)

`nn.Linear`: $Z^{[\ell]} = A^{[\ell-1]}W^T+b$
연산.

해당 layer의 

- 입력 차원 `n_input=30`
- 출력 차원 `n_output=60`



In이 30, Out이 60인nn

Column은 60

Row는 batch수

A는 30 W는 30을 60으로 바꿔줘야 하므로, 30 by 60

해당 과정 중요

In [23]:
# Example of nn.linear (In수, Out수, Batch는 알아서 됨. 어짜피 입력의 shape 0번이 batch사이즈)
linear_layer1 = nn.Linear(30, 60)

In [24]:
#test sample 생성해서 넣어줌
A = torch.randn(60, 30)
#60,30으로 넣어줘야 60 by 60나옴.. 곱해줄때, 전치되서 곱해짐
linear_layer1(A)

tensor([[-0.4969,  0.0564,  0.0403,  ...,  0.7190, -0.4373, -0.2725],
        [ 0.1313, -0.3374, -0.4736,  ...,  0.9976, -0.5047,  0.2178],
        [-0.1740, -1.1645, -0.4352,  ..., -0.0982, -0.6580,  0.6408],
        ...,
        [-0.3432,  0.3072,  0.7628,  ..., -0.3314,  0.2335,  0.1517],
        [ 0.5272,  0.3454, -0.5879,  ...,  0.7543,  0.7225,  0.2116],
        [ 0.3033, -0.1973,  0.3881,  ..., -0.1832,  1.0450,  0.7153]],
       grad_fn=<AddmmBackward0>)

In [25]:
linear_layer1(A).shape

torch.Size([60, 60])

근데 W 랑 B는 어디서??

linear_layer1에서 알아서 넣어줌

In [26]:
linear_layer1.weight

Parameter containing:
tensor([[-0.1186, -0.1156, -0.0194,  ...,  0.0404,  0.0339,  0.1559],
        [-0.1508,  0.1334,  0.0128,  ...,  0.1028,  0.0347, -0.0401],
        [ 0.0765, -0.1189,  0.0270,  ..., -0.1132,  0.0881, -0.0964],
        ...,
        [-0.0902, -0.0866,  0.0548,  ...,  0.1206, -0.0104, -0.0793],
        [-0.0640,  0.0198,  0.1421,  ...,  0.0484,  0.1701, -0.1360],
        [ 0.1803,  0.0207, -0.1754,  ...,  0.0740, -0.1130, -0.0051]],
       requires_grad=True)

+ How to get the weights and bias of each `nn.Linear`

In [27]:
linear_layer1.weight

Parameter containing:
tensor([[-0.1186, -0.1156, -0.0194,  ...,  0.0404,  0.0339,  0.1559],
        [-0.1508,  0.1334,  0.0128,  ...,  0.1028,  0.0347, -0.0401],
        [ 0.0765, -0.1189,  0.0270,  ..., -0.1132,  0.0881, -0.0964],
        ...,
        [-0.0902, -0.0866,  0.0548,  ...,  0.1206, -0.0104, -0.0793],
        [-0.0640,  0.0198,  0.1421,  ...,  0.0484,  0.1701, -0.1360],
        [ 0.1803,  0.0207, -0.1754,  ...,  0.0740, -0.1130, -0.0051]],
       requires_grad=True)

In [20]:
# Example of weights
linear_layer1.weight.data = torch.ones_like(linear_layer1.weight)

In [21]:
linear_layer1.weight 

Parameter containing:
tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]], requires_grad=True)

# NN example

- input units: 20
- hidden layer: 30, 40
- output units: 3
- activation function: ReLU
- output layer: No activation



In [52]:
# Simple NN construction

# 클래스로 선언
class FCN(nn.Module):#nn.Module 써야 함. => Base class for all NN modules.
    #constructor
    def __init__(self):
        super().__init__()#안쓰면 오류남. python문법 상속 클래스의 인스턴스를 임시로 불러옴, 그리고 실행.
        
        self.lin1 = nn.Linear(20, 30) #Input Hidden
        self.lin2 = nn.Linear(30, 40) #Hidden Hidden
        self.lin3 = nn.Linear(40, 3) #Hidden Output
        self.relu = nn.ReLU(True) #Use ReLU... 안해도 되는데 inplace연산해서 메모리 절약 가능. 디폴트는 False
        
    #forward의 장점 : model.함수명 안하고 그냥 model만 적어도 알아서 forward불러옴
    def forward(self, x): #
        x = self.lin1(x)
        x = self.relu(x)
        x = self.lin2(x)
        x = self.relu(x)
        x = self.lin3(x)
        return x
    


In [56]:
Xtrain = torch.randn(60, 20)

model = FCN()
model(Xtrain)
# = model.forward(Xtrain)

#만약 특정 레이어의 weight를 바꾸고 싶다면? 
#model.lin1.weight.data = torch.ones(30,20)
#model.lin1.weight.data = torch.ones_like(model.lin1.weight) 는 알아서 차원 맞춰주는데 숙제는 이거 쓰지 말고 직접

tensor([[-0.0509, -0.0498, -0.0223],
        [-0.1126,  0.0252,  0.0171],
        [-0.0315,  0.0152, -0.0474],
        [-0.0708, -0.0118, -0.0132],
        [-0.0566, -0.0194, -0.0388],
        [-0.1082,  0.0166, -0.0127],
        [-0.0202,  0.0185, -0.1269],
        [-0.1490, -0.0388, -0.0302],
        [-0.0678,  0.0493, -0.0363],
        [-0.0430, -0.0925, -0.0578],
        [-0.0670,  0.0326,  0.0037],
        [-0.1168, -0.0059, -0.0126],
        [-0.1966,  0.0233, -0.1207],
        [-0.0602,  0.0260, -0.0027],
        [-0.1000, -0.0419, -0.0509],
        [-0.1089, -0.0455, -0.0128],
        [-0.1287,  0.0223, -0.1286],
        [-0.0514,  0.0314,  0.0269],
        [ 0.0050, -0.0032, -0.1564],
        [-0.0479, -0.0459, -0.0974],
        [-0.1007, -0.0102, -0.0348],
        [-0.0536, -0.0622, -0.0419],
        [-0.0968, -0.0488, -0.1594],
        [-0.1790, -0.0703, -0.1127],
        [-0.0734, -0.0862, -0.1104],
        [-0.1266,  0.0027,  0.0104],
        [-0.0818,  0.0135, -0.0297],
 

In [57]:
model

FCN(
  (lin1): Linear(in_features=20, out_features=30, bias=True)
  (lin2): Linear(in_features=30, out_features=40, bias=True)
  (lin3): Linear(in_features=40, out_features=3, bias=True)
  (relu): ReLU(inplace=True)
)

In [34]:
# Example of parameters() in models
#파라메터 관련 내용..

# param_iterator = model.parameters()

# for param in param_iterator:
#     print(param)


In [35]:
# nn.Sequential() example

class FCN_seq(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.fc = nn.Sequential(nn.Linear(20, 30),
                      nn.ReLU(True),
                      nn.Linear(30, 40),
                      nn.ReLU(True),
                      nn.Linear(40, 3)
        )
        
    def forward(self, x):
        return self.fc(x)
        
        
        
# self.lin1 = nn.Linear(20, 30)
# self.lin2 = nn.Linear(30, 40)
# self.lin3 = nn.Linear(40, 3)
# self.relu = nn.ReLU(True)

In [36]:
model_seq = FCN_seq()
Xtrain.shape
model_seq

FCN_seq(
  (fc): Sequential(
    (0): Linear(in_features=20, out_features=30, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=30, out_features=40, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=40, out_features=3, bias=True)
  )
)

In [37]:
model_seq.fc[0]

Linear(in_features=20, out_features=30, bias=True)

In [38]:
class FCN_seq_v2(nn.Module):
    def __init__(self):
        super().__init__()
        

        
        temp = self.fcn_block(20, 30)+self.fcn_block(30, 40)+[nn.Linear(40,1)]
        self.fc = nn.Sequential(*temp)
        
        
    def fcn_block(self, in_dim, out_dim):
        return [nn.Linear(in_dim, out_dim),
                             nn.ReLU(True)]
        
        
    def forward(self, x):
        return self.fc(x)
        
        
        
# self.lin1 = nn.Linear(20, 30)
# self.lin2 = nn.Linear(30, 40)
# self.lin3 = nn.Linear(40, 3)
# self.relu = nn.ReLU(True)

In [39]:
model_seq_v2 = FCN_seq_v2()
model_seq_v2

FCN_seq_v2(
  (fc): Sequential(
    (0): Linear(in_features=20, out_features=30, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=30, out_features=40, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=40, out_features=1, bias=True)
  )
)

In [40]:
class FCN_final(nn.Module):
    def __init__(self, in_dim, hlayer, out_dim):
        super().__init__()
        
        l_list = self.fcn_block(in_dim, hlayer[0])
        
        for l1, l2 in zip(hlayer[:-1], hlayer[1:]):
            l_list = l_list + self.fcn_block(l1, l2)
        
        l_list = l_list + [nn.Linear(hlayer[-1], out_dim)]
        
        self.fc = nn.Sequential(*l_list)
        
        
    def fcn_block(self, in_dim, out_dim):
        return [nn.Linear(in_dim, out_dim),
                             nn.ReLU(True)]
        
        
    def forward(self, x):
        return self.fc(x)
        
        

In [41]:
hlayer = [30, 40]
in_dim = 20
out_dim= 3

myfcn_final = FCN_final(in_dim, hlayer, out_dim)

In [42]:
myfcn_final

FCN_final(
  (fc): Sequential(
    (0): Linear(in_features=20, out_features=30, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=30, out_features=40, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=40, out_features=3, bias=True)
  )
)

In [43]:
# Ordered dict example
# nn.Sequential() example

class FCN_seq_ordered_dic(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.fc = nn.Sequential(OrderedDict([('lin1', nn.Linear(20, 30)),
                      ('relu1', nn.ReLU(True)),
                      ('lin2', nn.Linear(30, 40)),
                      ('relu2',nn.ReLU(True)),
                      ('lin3', nn.Linear(40, 3))
                                            ])
        )
        
    def forward(self, x):
        return self.fc(x)
        
        
        
# self.lin1 = nn.Linear(20, 30)
# self.lin2 = nn.Linear(30, 40)
# self.lin3 = nn.Linear(40, 3)
# self.relu = nn.ReLU(True)

In [44]:
# ModuleList(), ModuleDict()

In [45]:
# state_dict() example
model.state_dict()

OrderedDict([('lin1.weight',
              tensor([[ 0.0299,  0.1571,  0.0047,  0.0533, -0.0884, -0.2145,  0.0765,  0.0220,
                       -0.0863,  0.1192,  0.1070,  0.0295,  0.1065, -0.0805,  0.0646,  0.1734,
                        0.0843,  0.0162,  0.0501, -0.0247],
                      [-0.1972, -0.1207,  0.1918, -0.0233,  0.2191, -0.0912,  0.1933,  0.1021,
                        0.1646,  0.2095, -0.0597, -0.0218,  0.1179,  0.0465, -0.1164, -0.1304,
                       -0.0958, -0.1624, -0.1906, -0.0230],
                      [-0.0408,  0.1159, -0.1523, -0.1142, -0.0331,  0.1372, -0.1167, -0.1371,
                        0.1438, -0.0618, -0.2210, -0.0483,  0.0356,  0.0835, -0.1710, -0.1157,
                       -0.0074, -0.0016, -0.0915,  0.0250],
                      [ 0.0668, -0.1925, -0.2236,  0.0358,  0.0248,  0.1855, -0.1504,  0.0093,
                       -0.0066,  0.0407, -0.1287,  0.0271,  0.0813, -0.2181,  0.0193, -0.2208,
                       -0.1587,

# Problem Setup

<div style="text-align:center"> <img src='https://drive.google.com/thumbnail?id=1FRhniwGeeutBSJQRdW6GzshMfDrPz7oJ&sz=w1000' width="250" height="200"> </div>
    
Build a Fully connected neural network with

- 3 layers
- 마지막 layer의 unit 수는 `1` 
  - 마지막 layer의 activation은 없음 (linear layer)
- Data feature 수는 `100`

- input unit 수는 data 크기를 보고 맞추세요
- hidden layer의 unit 수는 `[80, 50]`
  - hidden layer의 activation 함수는 ReLU

- model class 명 `myFCN`
  - instance 명 `my_model` 생성
  - `my_model` 출력

## Problem 1

problem setup에서 구성한 neural network을 `nn.Sequential`을 활용하여 생성하세요

In [109]:
## 사용할 data 
batch_size = 30
num_feature = 100

X_train = torch.randn(batch_size, num_feature)

In [110]:
X_train.shape

torch.Size([30, 100])

In [111]:
# Problem 1 코딩 (매 줄마다 주석 필수 )

# 클래스로 선언

'''
class myFCN(nn.Module):#nn.Module 써야 함. => Base class for all NN modules.
    #constructor
    def __init__(self):
        super().__init__()#안쓰면 오류남. python문법 상속 클래스의 인스턴스를 임시로 불러옴, 그리고 실행.
        
        
        self.lin1 = nn.Linear(100,80) #Input Hidden ,Data feature 수는 100
        self.lin2 = nn.Linear(80,50) #Hidden Hidden.. 80unit
        self.lin3 = nn.Linear(50,1) #Hidden Output..  50unit, 마지막 layer의 unit 수는 1
        self.relu = nn.ReLU(True) #inplace연산해서 메모리 절약, ReLU사용
        
    #forward의 장점 : model.함수명 안하고 그냥 model만 적어도 알아서 forward불러옴
    def forward(self, x): #
        x = self.lin1(x) #1번째 레이어 통과(In)
        x = self.relu(x) #활성화 함수
        x = self.lin2(x) #2번째 레이어 통솨
        x = self.relu(x) #활성화 함수
        x = self.lin3(x) #세번째 레이어 통과 (Out)
        #마지막 layer의 activation은 없음 (linear layer)
        return x #반환
'''
# nn.Sequential 사용    
class myFCN(nn.Module):
    #constructor
    def __init__(self):
        super().__init__()
        
        #nn.Sequential 사용하여 선언
        self.fc = nn.Sequential(
            nn.Linear(100,80), #1번째 레이어 100 > 80
            nn.ReLU(True),    #활성화 함수
            nn.Linear(80,50),  #2번째 레이어 80 > 50
            nn.ReLU(True),     ##활성화 함수
            nn.Linear(50,1)    #3번째 레이어 50 > 1
            #마지막 레이어는 활성화함수 없이
        )
        
    def forward(self, x):
        return self.fc(x) #위 과정 리턴


In [112]:
my_model = myFCN() #instance 명 my_model 생성

In [113]:
my_model#myModel의 형식

myFCN(
  (fc): Sequential(
    (0): Linear(in_features=100, out_features=80, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=80, out_features=50, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=50, out_features=1, bias=True)
  )
)

In [114]:
my_model(X_train)

tensor([[-0.0328],
        [-0.1639],
        [-0.1395],
        [-0.0641],
        [-0.1286],
        [ 0.0244],
        [-0.1074],
        [-0.0883],
        [-0.0153],
        [-0.0887],
        [-0.1091],
        [ 0.0849],
        [-0.0519],
        [-0.1736],
        [-0.1231],
        [-0.0475],
        [-0.1774],
        [-0.0641],
        [-0.0450],
        [-0.1203],
        [-0.0275],
        [-0.1464],
        [-0.0737],
        [-0.0183],
        [-0.1168],
        [-0.0458],
        [-0.0825],
        [-0.0201],
        [-0.0674],
        [ 0.0536]], grad_fn=<AddmmBackward0>)

## Problem 2

problem setup에서 구성한 neural network을 `OrderedDict`을 활용하여 생성하세요
- 각 layer의 이름을 주고 생성하세요

In [115]:
# 답작성

In [116]:
# nn.Sequential 과 OrderedDict 사용    
class myFCN(nn.Module):
    #constructor
    def __init__(self):
        super().__init__()
        
        #nn.Sequential 사용하여 선언, OrderedDict 사용하여 단계별 명명.
        self.fc = nn.Sequential(OrderedDict([
            ('layer_01', nn.Linear(100,80)), #1번째 레이어 100 > 80 레이어 이름 : layer_01
            ('ReLU_01', nn.ReLU(True)),    #활성화 함수             활성화함수 이름 : ReLU_01
            ('layer_02', nn.Linear(80,50)),  #2번째 레이어 80 > 50  레이어 이름 : layer_02
            ('ReLU_02',nn.ReLU(True)),     ##활성화 함수            활성화함수 이름 : ReLU_02
            ('layer_03', nn.Linear(50,1))    #3번째 레이어 50 > 1   레이어 이름 : layer_03
            #마지막 레이어는 활성화함수 없이
        ]))
        
    def forward(self, x):
        return self.fc(x) #위 과정 리턴

In [117]:
my_model = myFCN()

In [118]:
my_model

myFCN(
  (fc): Sequential(
    (layer_01): Linear(in_features=100, out_features=80, bias=True)
    (ReLU_01): ReLU(inplace=True)
    (layer_02): Linear(in_features=80, out_features=50, bias=True)
    (ReLU_02): ReLU(inplace=True)
    (layer_03): Linear(in_features=50, out_features=1, bias=True)
  )
)

In [119]:
my_model(X_train)

tensor([[-0.0878],
        [ 0.0639],
        [ 0.1186],
        [ 0.1359],
        [ 0.0436],
        [-0.0698],
        [ 0.0350],
        [-0.0652],
        [ 0.1491],
        [-0.0685],
        [ 0.0476],
        [ 0.0190],
        [ 0.0474],
        [-0.0610],
        [ 0.1055],
        [ 0.0520],
        [ 0.0893],
        [-0.0064],
        [-0.0300],
        [ 0.0254],
        [-0.0448],
        [-0.0139],
        [ 0.0117],
        [-0.0371],
        [-0.0651],
        [ 0.0353],
        [ 0.0020],
        [ 0.1054],
        [-0.0132],
        [ 0.0697]], grad_fn=<AddmmBackward0>)